# Búsqueda de autores en Orcid
See [search-orcid-registry](http://members.orcid.org/api/tutorial/search-orcid-registry)

ORCID: https://github.com/ORCID/python-orcid

In [1]:
%%writefile drive.cfg
[FILES]
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs 

Overwriting drive.cfg


In [2]:
import requests
import pandas as pd
import wosplus as wp

In [3]:
bdi=wp.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [4]:
bdi[:2]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación
1,71652139,Carlos Mario Jaramillo Ramirez,Carlos Mario,Jaramillo,Ramirez,Facultad de Artes,Departamento de Artes Visuales,DIDARTE. Didáctica de las artes


Obtener el token del API

In [5]:
# From https://github.com/ORCID/python-orcid
import orcid
api = orcid.PublicAPI('APP-R8J5P57TTE983P54', 'c6e1dd48-80d3-4dd9-9336-fc41728ac8fe', sandbox=False)
#token = api.get_token_from_authorization_code(authorization_code,
#                                              redirect_uri)


In [6]:
search_token = api.get_search_token_from_orcid()

In [7]:
search_token

'1d639b1a-4f0d-4189-89dc-84adbd9b1700'

https://pub.orcid.org

Comprobar que funciona

In [8]:
HEADERS_orcid={'Content-type': 'application/vnd.orcid+json',
    'Access token': search_token}

In [9]:
r=requests.get('https://pub.orcid.org/v2.1/search/?q=newman', headers=HEADERS_orcid)

Muestre los dos primeros elementos de la lista de resultados

In [11]:
pd.read_json(r.text)[:2]

,num-found,result
0,924,"{'orcid-identifier': {'path': '0000-0002-5662-263X', 'uri': 'https://orcid.org/0000-0002-5662-263X', 'host': 'orcid.org'}}"
1,924,"{'orcid-identifier': {'path': '0000-0002-7182-6999', 'uri': 'https://orcid.org/0000-0002-7182-6999', 'host': 'orcid.org'}}"


Definir la función de búsqueda. Ver ayuda en la documentación del API

In [12]:
import pandas as pd
import time
import random
def search_orcid(q='family-name:Taylor+AND+given-names:Michael',search_token=''):
    '''
    Search API orcid in json for query q: (see examples here: http://members.orcid.org/api/tutorial/search-orcid-registry)
    Example: 
    q=family-name:Taylor+AND+given-names:Michael
    '''
    HEADERS_orcid={'Content-type': 'application/vnd.orcid+json',
    'Access token': search_token}
    r=requests.get('https://pub.orcid.org/v2.1/search/?q={}'.format(q),
                   headers=HEADERS_orcid)
    
    # wait for next request
    time.sleep(random.randint(1,3))
    if r.status_code==200:
        return pd.read_json(r.text)
    else:
        return pd.DataFrame()

## Formulas de búsqueda

Muestra de búsqueda por nombre para investigador de la UdeA

In [14]:
df=search_orcid('family-name:Restrepo+AND+given-names:Diego+AND+affiliation-org-name:Universidad+de+Antioquia',
                search_token)#+AND+given-names:Diego')#+AND+affiliation-org-name:Universidad+de+Antioquia')

Muestre los primeros dos resultados de la búsqueda

In [15]:
df[:2]

,num-found,result
0,6,"{'orcid-identifier': {'path': '0000-0001-6455-5564', 'uri': 'https://orcid.org/0000-0001-6455-5564', 'host': 'orcid.org'}}"
1,6,"{'orcid-identifier': {'path': '0000-0002-9054-3847', 'uri': 'https://orcid.org/0000-0002-9054-3847', 'host': 'orcid.org'}}"


### Prepare búsqueda investigadores UdeA

In [17]:
apellido='Restrepo'
nombre='Diego'
affiliation='Universidad de Antioquia'

In [18]:
df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

In [19]:
df[:2]

,num-found,result
0,6,"{'orcid-identifier': {'path': '0000-0001-6455-5564', 'uri': 'https://orcid.org/0000-0001-6455-5564', 'host': 'orcid.org'}}"
1,6,"{'orcid-identifier': {'path': '0000-0002-9054-3847', 'uri': 'https://orcid.org/0000-0002-9054-3847', 'host': 'orcid.org'}}"


Además de afiliación:
1. Nombres y apellidos completo
2. Primer nombre y primer apellido
3. Segundo nombre y primer appellido

Se va a hacer un loop sobre la base de datos de investigadores de la UdeA. Se ilustrará primero lo que pasa con la primera entrada  de la lista cuando se búsca de la forma 1.

In [20]:
bdi[:1]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación


In [21]:
i=0

In [22]:
bdi.loc[i,'NOMBRES'].strip()

'Bernardo Maria'

In [23]:
bdi.loc[i,'PRIMER APELLIDO']+' '+bdi.loc[i,'SEGUNDO APELLIDO']

'Bustamante Cardona'

In [24]:
apellido=bdi.loc[i,'PRIMER APELLIDO'].strip()
apellido_completo=apellido+' '+bdi.loc[i,'SEGUNDO APELLIDO'].strip()
nombre_completo=bdi.loc[i,'NOMBRES'].strip()
affiliation='Universidad de Antioquia'
df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido_completo.replace(' ','+'), 
                  nombre_completo.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

In [26]:
df

,num-found,result


Ahora intentamos con la búsqueda 2. 

Para ello añadimos una columna con el primer nombre al DataFrame

In [30]:
bdi['Primer_NOMBRE']=bdi.NOMBRES.str.split().str[0]
nombre=bdi.loc[i,'Primer_NOMBRE'].strip()

In [31]:
bdi[:1]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO,Primer_NOMBRE
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación,Bernardo


In [33]:
df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), 
                  nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

In [34]:
df

,num-found,result


Finalmente intentamos con la búsqueda 3.

In [35]:
bdi['Segundo_NOMBRE']=bdi.NOMBRES.str.split().str[1].fillna('')
segundo_nombre=bdi.loc[i,'Segundo_NOMBRE'].strip()
segundo_nombre

'Maria'

In [36]:
df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), 
                  segundo_nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

In [37]:
df

,num-found,result
0,1,"{'orcid-identifier': {'path': '0000-0003-3253-1597', 'uri': 'https://orcid.org/0000-0003-3253-1597', 'host': 'orcid.org'}}"


In [38]:
dft=pd.DataFrame()
affiliation='Universidad de Antioquia'
# Adicionar dos columnas nuevas al DataFrame para las búsquedas 2 y 3:
bdi['Primer_NOMBRE']=bdi.NOMBRES.str.split().str[0]
bdi['Segundo_NOMBRE']=bdi.NOMBRES.str.split().str[1].fillna('')
for i in bdi.index:
    df=pd.DataFrame()
    print(i)
    nombre_completo=bdi.loc[i,'NOMBRES'].strip()
    nombre=bdi.loc[i,'Primer_NOMBRE'].strip()
    segundo_nombre=bdi.loc[i,'Segundo_NOMBRE'].strip()
    apellido=bdi.loc[i,'PRIMER APELLIDO'].strip()
    apellido_completo=apellido+' '+bdi.loc[i,'SEGUNDO APELLIDO'].strip()
    #1. Busqueda en completo
    df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido_completo.replace(' ','+'), 
                  nombre_completo.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)
    if not df.empty:
        df['forma']=1
    
    #2. Búsqueda de primer nombre y primer appellido
    if df.empty:
        df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), 
                  nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)
        if not df.empty:
            df['forma']=2

        #3. Búsqueda de segundo nombre y primer appellido     
        if df.empty and segundo_nombre:
            df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), 
                  segundo_nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)
            if not df.empty:
                df['forma']=3        

    if i==5:
        break
                
    if df.empty:
        continue 
    else:
        #inentar verificar y si funciona:
        df['NOMBRE COMPLETO']=bdi.loc[i,'NOMBRE COMPLETO'].strip()
        dft=dft.append(df.loc[[0]]).reset_index(drop=True)
        


0
1
2
3
4
5


In [41]:
bdi[:5]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO,Primer_NOMBRE,Segundo_NOMBRE
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación,Bernardo,Maria
1,71652139,Carlos Mario Jaramillo Ramirez,Carlos Mario,Jaramillo,Ramirez,Facultad de Artes,Departamento de Artes Visuales,DIDARTE. Didáctica de las artes,Carlos,Mario
2,43613390,Isabel Cristina Restrepo Acevedo,Isabel Cristina,Restrepo,Acevedo,Facultad de Artes,Departamento de Artes Visuales,Hipertrópico: Convergencia entre Arte y Tecnología,Isabel,Cristina
3,43986489,Alexandra Milena Tabares Garcia,Alexandra Milena,Tabares,Garcia,Facultad de Artes,Departamento de Artes Visuales,Hipertrópico: Convergencia entre Arte y Tecnología,Alexandra,Milena
4,15257718,Bernardo Barragan Castrillon,Bernardo,Barragan,Castrillon,Facultad de Artes,Departamento de Artes Visuales,Historia de la Práctica Pedagógica en Colombia,Bernardo,


In [79]:
dft[:5]

,num-found,result,forma,NOMBRE COMPLETO
0,1,"{'orcid-identifier': {'path': '0000-0003-3253-1597', 'uri': 'https://orcid.org/0000-0003-3253-1597', 'host': 'orcid.org'}}",3,Bernardo Maria Bustamante Cardona
1,2,"{'orcid-identifier': {'path': '0000-0003-4969-8457', 'uri': 'https://orcid.org/0000-0003-4969-8457', 'host': 'orcid.org'}}",1,Carlos Mario Jaramillo Ramirez
2,1,"{'orcid-identifier': {'path': '0000-0002-8235-3525', 'uri': 'https://orcid.org/0000-0002-8235-3525', 'host': 'orcid.org'}}",1,Isabel Cristina Restrepo Acevedo


In [80]:
dft['result'].map(lambda x: x['orcid-identifier']).map(lambda x: x['uri'])

0    https://orcid.org/0000-0003-3253-1597
1    https://orcid.org/0000-0003-4969-8457
2    https://orcid.org/0000-0002-8235-3525
Name: result, dtype: object

## Appendix

In [17]:
r=requests.get('http://docs.python-requests.org/en/master/')

In [23]:
print( r.text[:193] )


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
  <head>
    
